In [1]:
from otx.engine import Engine

config = "../src/otx/recipe/classification/multi_class_cls/otx_efficientnet_b0.yaml"
data_root = "../tests/assets/classification_dataset"

engine = Engine.from_config(
    config_path=config,
    data_root=data_root,
)

/home/harimkan/workspace/repo/otx-regression/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The corresponding keys in config are not used.: ['verbose', 'data_root', 'task', 'seed', 'callback_monitor', 'resume', 'disable_infer_num_classes']


init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip
init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip
init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip


In [2]:
engine.train(max_epochs=2)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params
---------------------------------------------------------------
0 | model       | EfficientNetB0ForMulticlassCls | 5.6 M 
1 | val_metric  | MulticlassAccuracy             | 0     
2 | test_metric | MulticlassAccuracy             | 0     
---------------------------------------------------------------
5.6 M     Trainable params
0         Non-trainable params
5.6 M     Total param

/home/harimkan/workspace/repo/otx-regression/venv/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:293: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 1: 100%|██████████| 1/1 [00:00<00:00,  9.88it/s, v_num=6, train/loss=0.684, val/accuracy=0.720]

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 1/1 [00:00<00:00,  5.67it/s, v_num=6, train/loss=0.684, val/accuracy=0.720]


{'train/loss': tensor(0.6839), 'val/accuracy': tensor(0.7200)}

In [3]:
engine.test()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 61.77it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/accuracy       │    0.5600000023841858     │
└───────────────────────────┴───────────────────────────┘

{'test/accuracy': tensor(0.5600)}

In [4]:
exported_model_path = engine.export()
exported_model_path

PosixPath('otx-workspace/exported_model.xml')

In [5]:
# Current Way - Case
from otx.core.model.entity.classification import OVMulticlassClassificationModel

# User Need to create OVMulticlassClassificationModel for OV testing
ov_model = OVMulticlassClassificationModel(
    model_name=exported_model_path,
    num_classes=engine.datamodule.meta_info.num_classes,
    model_type="Classification",
)

# Need to create another engine for OV test
ov_engine = Engine(
    data_root=data_root,
    model=ov_model,
)

ov_engine.test()

/home/harimkan/workspace/repo/otx-regression/src/otx/core/utils/build.py:52: UserWarning: Set the default number of OpenVINO inference requests to 8.
            You can specify the value in config.
  warnings.warn(msg, stacklevel=1)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 12.31it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/accuracy       │    0.5600000023841858     │
└───────────────────────────┴───────────────────────────┘

{'test/accuracy': tensor(0.5600)}

In [6]:
# Suggestion Way

# Using Exsting Engine for OV Testing
# How about we make it simple with auto-configuration?
engine.test(checkpoint=exported_model_path)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 12.28it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/accuracy       │    0.5600000023841858     │
└───────────────────────────┴───────────────────────────┘

{'test/accuracy': tensor(0.5600)}